# Categorization Playground
This playground is used for manual testing the categorization Service

### Constants

In [32]:
CATEGORIZATION_API_CLASSIFY = 'http://localhost:8002/classify'
CATEGORIZATION_API_QA = 'http://localhost:8002/qa'
TRANSCRIPTION_PATH = "/shared/media/calls/base_call.txt"
CATEGORIES = [
    {
        "title": "Visa and Passport Services",
        "points": ["Visa application procedures", "Passport renewal", "Required documents for applications", "Document processing times", "Visa fees and payments", "Availability of expedited processing"]
    },
    {
        "title": "Diplomatic Inquiries",
        "points": ["Political issues between countries", "Diplomatic exchanges", "International agreements and treaties", "Resolution of international conflicts", "Cooperation in the field of security"]
    },
    {
        "title": "Travel Advisories",
        "points": ["Security warnings for destination countries", "Information on natural disasters", "Updates on epidemic situations", "Health and vaccination recommendations", "Local laws and customs guidelines"]
    },
    {
        "title": "Consular Assistance",
        "points": ["Assistance to citizens abroad", "Issuance of powers of attorney and documents", "Support in case of arrest or detention", "Fine remission and legal assistance", "Repatriation and emergency return"]
    },
    {
        "title": "Trade and Economic Cooperation",
        "points": ["Investment opportunities", "Trade agreements and regulations", "Cooperation in technology and innovation", "Support for small and medium enterprises", "Market analysis and economic trends"]
    }
]
VISA_AND_PASSPORT_SERVICES_DIALOGUE = """I'm planning to move to Canada for work and need to understand the visa application process. Can you guide me through the necessary steps?
Certainly! To apply for a Canadian work visa, you'll need a valid job offer from a Canadian employer, a completed application form, your current passport, proof of qualifications, and any required certificates. Additionally, you'll need to undergo a medical examination and obtain a police clearance certificate. The processing time can vary, so it's advisable to apply well in advance. You can also explore expedited processing options if available. For detailed instructions and to submit your application, please visit our official website or contact our visa support team."""

In [2]:
import time
import random
import string

import requests

### Functions

In [3]:
def get_candidate_labels(categories=CATEGORIES):
    for category in categories:
        yield category["title"]
        for point in category["points"]:
            yield point

In [20]:
def make_classify_request(call_id, labels, context=None, context_path=None):
    data = {"call_id": call_id, "candidate_labels": labels}
    
    if context:
        data["context"] = context
    elif context_path:
        data["context_path"] = context_path
    
    return requests.post(CATEGORIZATION_API_CLASSIFY, json=data)

visa_call = lambda prefix="": make_classify_request(prefix + "visa_call", context=VISA_AND_PASSPORT_SERVICES_DIALOGUE, labels=list(get_candidate_labels()))
transcription_call = lambda prefix="": make_classify_request(prefix + "transcription_call", context_path=TRANSCRIPTION_PATH, labels=list(get_candidate_labels()))

In [37]:
def make_qa_request(call_id, question, context=None, context_path=None):
    data = {"call_id": call_id, "question": question}
    
    if context:
        data["context"] = context
    elif context_path:
        data["context_path"] = context_path
        
    print(data)
    
    return requests.post(CATEGORIZATION_API_QA, json=data)

NAME_QUESTION = "Get the name of the person who is makes the call?"
LOCATION_QUESTION = "Name the location {0}?"

visa_name_qa = lambda prefix="": make_qa_request(prefix + "visa_name_qa", NAME_QUESTION, context=VISA_AND_PASSPORT_SERVICES_DIALOGUE)
visa_location_qa = lambda prefix="": make_qa_request(prefix + "visa_location_qa", LOCATION_QUESTION.format("of the person who makes a call"), context=VISA_AND_PASSPORT_SERVICES_DIALOGUE)
transcription_name_qa = lambda prefix="": make_qa_request(prefix + "transcription_name_qa", NAME_QUESTION, context_path=TRANSCRIPTION_PATH)
transcription_location_qa = lambda prefix="": make_qa_request(prefix + "transcription_location_qa", LOCATION_QUESTION.format("of the person who makes a call"), context_path=TRANSCRIPTION_PATH)

In [28]:
def show_execution_time(method, *args, **kwargs):
    start = time.time()
    result = method(*args, **kwargs)
    print(f"Execution time: {time.time() - start}")
    return result

In [29]:
def random_string(length=10):
    return ''.join(random.choices(string.ascii_uppercase + string.digits, k=length))

### Manual Tests (Classify)

In [30]:
request = show_execution_time(visa_call, random_string())
print(request.status_code)
print(request.json())

Execution time: 0.00724029541015625
200
{'message': 'Zero Shot Classification job submitted.'}


In [23]:
request = show_execution_time(transcription_call, random_string())
print(request.status_code)
print(request.json())

Execution time: 0.01736307144165039
200
{'message': 'Classification job submitted.'}


### Manual Tests (QA)

In [38]:
request = show_execution_time(visa_name_qa, random_string())
print(request.status_code)
print(request.json())

request = show_execution_time(visa_location_qa, random_string())
print(request.status_code)
print(request.json())


{'call_id': 'BSW43JSIJWvisa_name_qa', 'question': 'Get the name of the person who is makes the call?', 'context': "I'm planning to move to Canada for work and need to understand the visa application process. Can you guide me through the necessary steps?\nCertainly! To apply for a Canadian work visa, you'll need a valid job offer from a Canadian employer, a completed application form, your current passport, proof of qualifications, and any required certificates. Additionally, you'll need to undergo a medical examination and obtain a police clearance certificate. The processing time can vary, so it's advisable to apply well in advance. You can also explore expedited processing options if available. For detailed instructions and to submit your application, please visit our official website or contact our visa support team."}
Execution time: 0.005999088287353516
422
{'detail': [{'type': 'int_parsing', 'loc': ['body', 'call_id'], 'msg': 'Input should be a valid integer, unable to parse stri